In [1]:
from sklearn.ensemble.forest import RandomForestRegressor
import pandas as pd
import numpy as np
from helpers import add_datepart, train_cats, proc_df
from IPython.display import display
import math

/home/mayorov/.virtualenvs/fastai/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
        display(df)

In [3]:
df_raw = pd.read_csv("data/bulldozers/Train.csv", parse_dates=['saledate'], low_memory=False)
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [4]:
df_raw

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,9.259131,1840702,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,9.305651,1830472,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,9.350102,1887659,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9.104980,1903570,21435,149,2.0,2005,NaN,NaN,2011-10-25,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


```
m = RandomForestRegressor(n_jobs=-1)
m.fit(df_raw.drop("SalePrice", axis=1), df_raw.SalePrice)
```

Этот код не сработает по той причине, что мы не провели никакого Feature Engineering. Большинство алгоритмов машинного обучения может работать только с численными данными. Таким образом, например, категориальные переменные нам надо конвертировать из строк в численные категории. Гораздо интереснее что можно выжать из поля saledate - год, месяц, день, выходной ли это, начало(конец) ли года или квартала, как минимум. 

Для этого я воспользуюсь функцией из старой версии fastai, просто для удобства. Ничего мистического, просто мы берем стандартные атрибуты поля типа datetime в pandas и создаем из них новые колонки, исходную колонку дропаем.

In [5]:
add_datepart(df_raw, "saledate")
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

Как было сказано выше, строки при обучении не несут никакой полезной информации, нужно конвертировать их в специальный тип данных pandas - категории. Для этого снова возьмем готовый хелпер, но он еще проще - представление строк в виде категорий это стандартный метод pandas.

Эти изменения необходимы нам только на этапе тренировки модели.

In [6]:
train_cats(df_raw)

In [7]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [8]:
df_raw.UsageBand.cat.codes

0         1
1         1
2         0
3         0
4         2
         ..
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Length: 401125, dtype: int8

Последняя трудность - отсутствующие значения, посмотрим долю пропущенных по каждой колонке:

In [9]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

Пропущенные значения можно обрабатывать разными способами, самый распространенный - заменять их на медианное значение по столбцу. Полезно так же иметь отдельную колонку с булевскими флажками и помечать, что этого значения не было и оно искусственное.

Для этого так же есть небольшой хелпер, который к тому же отделить для нас зависимую переменную:

In [10]:
df, y, nas = proc_df(df_raw, 'SalePrice')

Теперь, после предобработки датасета, наш исходный код должен сработать

In [11]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df, y)

0.9881389725397574

При таком хорошем результате возникает подозрение, что мы переобучились. Стандартной практикой является наличие валидационного сета, который не участвует в тренировке, а только в проверке результатов модели.

Отделим от тренировочного сета столько же значений для валидации, сколько использует кагл для проверки этой задачи, то есть 12к записей

In [12]:
def split_vals(a, n):
    return a[:n].copy(), a[n:].copy()

n_valid = 12000
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
x_train, x_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

x_train.shape, y_train.shape, x_valid.shape 

((389125, 66), (389125,), (12000, 66))

In [13]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x_train), y_train), rmse(m.predict(x_valid), y_valid),
          m.score(x_train, y_train), m.score(x_valid, y_valid)]
    print(res)

In [14]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(x_train, y_train)
print_score(m)

CPU times: user 13min 43s, sys: 2.93 s, total: 13min 46s
Wall time: 3min 37s
[0.07569409838361545, 0.23405589119030368, 0.9880254592365818, 0.9021666043294357]


Как работает одно дерево? На каждом шаге мы проверяем все возможные переменные на все возможные бинарные деления и выбираем лучшее. После каждого разбиения мы имеем два сета и соответственно две метрики. Чтобы понять, что разбиение лучшее, надо взять среднее взвешенное этих метрик по количеству сэмплов в сэтах. Лучшее разбиение - максимальное улучшение этой взвешенной метрики по двум подсетам. 

Процесс продолжается, пока в листьях не останется по одному сэмплу или пока не достигнуто какое-то искусственное ограничение (глубина дерева, например, один из параметров класса RandomForestRegressor).

Так как полный датасет обрабатывается достаточно долгое время - около 4 минут на этой конкретной машине, - для быстрых экспериментов возьмем сабсет поменьше, валидационный сет оставим прежним и менять его в ходе экспериментов не будем, чтобы можно было отслеживать относительную результативность моделей без случайности.

In [23]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset=30000, na_dict=nas)
x_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

Попробуем результаты регрессора с единственным маленьким деревом

In [25]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(x_train, y_train)
print_score(m)

[0.5269748999549773, 0.5811097475618197, 0.3951014846316633, 0.3969347630685759]


Если убрать ограничение по глубине дерева - результаты улучшатся, но на валидации будут все равно хуже оригинальной модели. Score на тренировочном сете - единица, наше дерево идеально предсказывает значения тренировочного сета, но плохо справляется с новыми данными.

In [27]:
m = RandomForestRegressor(n_estimators=1, bootstrap=False, n_jobs=-1)
m.fit(x_train, y_train)
print_score(m)

[4.351167857633658e-17, 0.5955195848657648, 1.0, 0.36665540092687554]


46:00